In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
import math

data = pd.read_csv("dataAfterPOS.csv")
#remove description
data['description'] = data['description'].str.replace('[^\w\s]','')

# Pre processing (always good for everything you do): MUST do

In [2]:
#remove stopwords 
from nltk.corpus import stopwords
stop = stopwords.words('english')

data["stopwords"] = data["description"].apply(lambda x: len([x for x in x.split() if x in stop]))

In [3]:
#remove most frequent and less frequent bcs useless
freq = pd.Series(' '.join(data['description']).split()).value_counts()[:10]
freq = list(freq.index)
data['description'] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

unfreq = pd.Series(' '.join(data["description"]).split()).value_counts()
count = 0
for el in unfreq: 
    if(el < 10):
        count = count + 1
print(count)   

30993


In [4]:
unfreqList = unfreq[-count:]
unfreq = dict(unfreqList)  
data["description"] = data['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

# Tf + idf + tf-idf on whole vanilla dataset (no stemming = "basic" words)

In [5]:
data.drop(["word_count","stopwords"],axis=1, inplace=True)
fullText = pd.Series(' '.join(data['description']).split()).value_counts()
#getValueCounts
tfData = {}
for idx, row in data.iterrows(): 
    tfData[idx] = pd.Series(row['description'].split()).value_counts();

In [6]:
#compute tf for every word
tf1 = {}
for idx in tfData.keys():
    tf1[idx] = {}
    for word in tfData[idx].keys():
        tf1[idx][word] = tfData[idx][word] / fullText[word]

In [7]:
tf1[0]

{'brisk': 0.0005434782608695652,
 'herb': 0.00017717930545712261,
 'sage': 0.0006402048655569782,
 'apple': 8.802816901408451e-05,
 'brimstone': 0.01818181818181818,
 'citrus': 0.00010035122930255896,
 'tropical': 0.000310077519379845,
 'expressive': 0.0018315018315018315,
 'dried': 0.00034411562284927734,
 'broom': 0.0058823529411764705}

In [16]:
tf1[1]

{'red': 6.177033788374823e-05,
 'fruits': 8.220304151253596e-05,
 'fruity': 0.00011640088464672332,
 'juicy': 0.00011844131232974062,
 'berry': 7.289692374981776e-05,
 'drinkable': 0.0012690355329949238,
 'firm': 0.00011984659635666346}

If you want a dataset on the whole text, without grouping:

In [12]:
desc = []
for idx, row in data.iterrows():
    modded = []
    temp = row["description"].split()
    for word in temp:
        modded.append(tf1[idx][word])
    desc.append(modded)
#desc[0]

[0.000310077519379845,
 0.0058823529411764705,
 0.01818181818181818,
 0.00017717930545712261,
 0.0018315018315018315,
 8.802816901408451e-05,
 0.00010035122930255896,
 0.00034411562284927734,
 0.0006402048655569782,
 0.0005434782608695652]

In [14]:
data["description_values_tf"] = desc 

In [15]:
data.to_csv()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,word_count,stopwords,description_values_tf
0,Italy,tropical broom brimstone herb expressive apple...,Vulkà Bianco,87,17.000000,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24,0,"[0.000310077519379845, 0.0058823529411764705, ..."
1,Portugal,fruity firm juicy red berry fruits drinkable,Avidagos,87,15.000000,Douro,unknown,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39,0,"[0.00011640088464672332, 0.0001198465963566634..."
2,US,tart snappy lime flesh dominate green pineappl...,unknown,87,14.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28,0,"[0.00021204410517387616, 0.0017921146953405018..."
3,US,pineapple rind lemon pith orange blossom opule...,Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33,0,"[0.00030147723846849563, 0.001326259946949602,..."
4,US,much regular bottling rough tannic rustic eart...,Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41,0,"[0.0005330490405117271, 0.003003003003003003, ..."
5,Spain,blackberry typical navarran whiff green herbs ...,Ars In Vitro,87,15.000000,Northern Spain,Navarra,unknown,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,42,0,"[9.013881377321075e-05, 0.0011918951132300357,..."
6,Italy,bright informal candied berry white pepper sav...,Belsito,87,16.000000,Sicily & Sardinia,Vittoria,unknown,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,31,0,"[0.00010972130787798991, 0.003115264797507788,..."
7,France,dry restrained offers profusion firm texture m...,unknown,87,24.000000,Alsace,Alsace,unknown,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,21,0,"[7.183908045977011e-05, 0.0034965034965034965,..."
8,Germany,savory thyme notes accent preserved peach bris...,Shine,87,12.000000,Rheinhessen,unknown,unknown,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,24,0,"[0.0002176278563656148, 0.0010976948408342481,..."
9,France,great depth flavor fresh apple pear fruits tou...,Les Natures,87,27.000000,Alsace,Alsace,unknown,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,30,0,"[0.0002175805047867711, 0.00044169611307420494..."


In [ ]:
#idf
docCount = dict.fromkeys(fullText.keys(),0)
idfDict1 = {}
N = len(data)                                #numero di documenti #dizionario contenente tutte le parole presenti 
idfData = dict(tfData)
for el in idfData:     #per ogni descrizione
    for word,val in idfData[el].items():
        docCount[word] = docCount[word] + 1
        
for word in fullText.keys():
    idfDict1[word] = math.log10(N/docCount[word])  